# Single Event with Complex Analysis

## Part Two (ML Training)

#### Prerequisites
This notebook is designed to work with a Stroom server process running on `localhost`, into which data from `EventGen` application has been ingested and indexed in the manner described in `stroom-analytic-demo`.

You must set the environmental variable `STROOM_API_KEY` to the API token associated with a suitably privileged Stroom user account before starting the Jupyter notebook server process.

In [21]:
from pyspark.sql.types import *
from pyspark.sql.functions import from_json, col, coalesce, unix_timestamp,lit,to_timestamp,hour,date_format,date_trunc
from pyspark.ml.feature import OneHotEncoderEstimator,VectorAssembler,StringIndexer
from pyspark.ml.regression import LinearRegression,RandomForestRegressor
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from IPython.display import display
import time,os

#### Schema Discovery
It is necessary to specify the structure of the JSON data arriving on the topic.  This structure can be determined at runtime.

As the same format of data is also available via an indexed search using the `stroom-spark-datasource`, one way to determine the JSON schema is by interrogating the data held in the `Sample Index` Stroom index.

The specified pipeline is a Stroom Search Extraction Pipeline that uses the stroom:json XSLT function to create a JSON representation of the entire event.  This field is called "Json" by default but the name of the field that contains the JSON representation can (optionally) be changed with the parameter jsonField.

In this manner, all data is returned as a single JSON structure within the field **json**

In [10]:
spark = SparkSession \
    .builder \
    .appName("MyTestApp") \
    .getOrCreate()
schemaDf = spark.read.format('stroom.spark.datasource.StroomDataSource').load(
        token=os.environ['STROOM_API_KEY'],host='localhost',protocol='http',
        uri='api/stroom-index/v2',traceLevel="0",
        index='32dfd401-ee11-49b9-84c9-88c3d3f68dc2',pipeline='13143179-b494-4146-ac4b-9a6010cada89',
        maxResults='300000').filter((col('idxEventTime') > '2018-01-01T00:00:00.000Z')
            & (col('idxEventTime') < '2019-01-01T00:00:00.000Z')
            & (col('idxDescription') == 'Authentication Failure'))

print ('Using ', schemaDf.count(), ' records for training')
json_schema = spark.read.json(schemaDf.rdd.map(lambda row: row.json)).schema

json_schema

Using  56889  records for training


StructType(List(StructField(EventDetail,StructType(List(StructField(Authenticate,StructType(List(StructField(Action,StringType,true),StructField(Outcome,StructType(List(StructField(Permitted,StringType,true),StructField(Reason,StringType,true),StructField(Success,StringType,true))),true),StructField(User,StructType(List(StructField(Id,StringType,true))),true))),true),StructField(TypeId,StringType,true))),true),StructField(EventId,StringType,true),StructField(EventSource,StructType(List(StructField(Device,StructType(List(StructField(HostName,StringType,true))),true),StructField(Generator,StringType,true),StructField(System,StructType(List(StructField(Environment,StringType,true),StructField(Name,StringType,true))),true))),true),StructField(EventTime,StructType(List(StructField(TimeCreated,StringType,true))),true),StructField(StreamId,StringType,true)))

## Feature Engineering
Creating a feature vector suitable for ML

In [11]:
featuresDf = schemaDf.withColumn('evt', from_json(col('json'), json_schema)).\
    withColumn ('timestamp', to_timestamp(col('evt.EventTime.TimeCreated')).cast("timestamp")).\
    where(col("timestamp") < '2019-01-01').\
    withColumn('operation', col('evt.EventDetail.TypeId')).\
    groupBy(date_trunc('day',"timestamp").alias("date"), 
            date_format('timestamp', 'EEEE').alias("day"), 
            hour("timestamp").alias("hour"),
            col('operation')).\
    count().sort(col('date'),col('hour'))


# operationNameIndexer = StringIndexer(inputCol="operation",outputCol="opCat")
# operationEncoder = OneHotEncoderEstimator(inputCols=['opCat'],outputCols=['opVec'])
hourEncoder = OneHotEncoderEstimator(inputCols=['hour'],outputCols=['hourVec'])
dayNameIndexer = StringIndexer(inputCol="day",outputCol="dayCat")
dayEncoder = OneHotEncoderEstimator(inputCols=['dayCat'],outputCols=['dayVec'])
basicPipeline = Pipeline(stages=[hourEncoder, dayNameIndexer, dayEncoder])

pipelineModel = basicPipeline.fit(featuresDf)
pipelineModel.write().overwrite().save("models/inputVecPipelineModel")

vecDf = pipelineModel.transform(featuresDf)

vecDf.show()


+-------------------+-------+----+--------------------+-----+---------------+------+-------------+
|               date|    day|hour|           operation|count|        hourVec|dayCat|       dayVec|
+-------------------+-------+----+--------------------+-----+---------------+------+-------------+
|2018-01-01 00:00:00| Monday|  12|Authentication Fa...|   11|(23,[12],[1.0])|   4.0|(6,[4],[1.0])|
|2018-01-01 00:00:00| Monday|  13|Authentication Fa...|   33|(23,[13],[1.0])|   4.0|(6,[4],[1.0])|
|2018-01-01 00:00:00| Monday|  14|Authentication Fa...|   15|(23,[14],[1.0])|   4.0|(6,[4],[1.0])|
|2018-01-01 00:00:00| Monday|  15|Authentication Fa...|   24|(23,[15],[1.0])|   4.0|(6,[4],[1.0])|
|2018-01-01 00:00:00| Monday|  16|Authentication Fa...|   11|(23,[16],[1.0])|   4.0|(6,[4],[1.0])|
|2018-01-01 00:00:00| Monday|  17|Authentication Fa...|   10|(23,[17],[1.0])|   4.0|(6,[4],[1.0])|
|2018-01-01 00:00:00| Monday|  18|Authentication Fa...|   14|(23,[18],[1.0])|   4.0|(6,[4],[1.0])|
|2018-01-0

We can now create the entire feature vector.  Shown below with what will be the required output vector (actually a simple scalar "count")

In [12]:
vectorAssembler = VectorAssembler(inputCols = ['hourVec','dayVec'], outputCol = 'features')

trainingDf = vectorAssembler.transform(vecDf).select('features','count')

trainingDf.show()


+--------------------+-----+
|            features|count|
+--------------------+-----+
|(29,[12,27],[1.0,...|   11|
|(29,[13,27],[1.0,...|   33|
|(29,[14,27],[1.0,...|   15|
|(29,[15,27],[1.0,...|   24|
|(29,[16,27],[1.0,...|   11|
|(29,[17,27],[1.0,...|   10|
|(29,[18,27],[1.0,...|   14|
|(29,[19,27],[1.0,...|   11|
|(29,[20,27],[1.0,...|    5|
|(29,[21,27],[1.0,...|    2|
|(29,[22,27],[1.0,...|    2|
|     (29,[27],[1.0])|    3|
|(29,[0,25],[1.0,1...|    2|
|(29,[1,25],[1.0,1...|    1|
|(29,[2,25],[1.0,1...|    2|
|(29,[3,25],[1.0,1...|    3|
|(29,[4,25],[1.0,1...|    1|
|(29,[5,25],[1.0,1...|    1|
|(29,[7,25],[1.0,1...|    2|
|(29,[8,25],[1.0,1...|    8|
+--------------------+-----+
only showing top 20 rows



## Training (Linear Regression)
Now create a Linear Regression to predict the number of auth failures in each hour/day of week.
Save the model for later use.

In [13]:
linearReg = LinearRegression(maxIter=20, regParam=0.001, featuresCol='features', labelCol='count')

linearRegModel = linearReg.fit(trainingDf)

linearRegModel.write().overwrite().save("models/linearRegressionAuthFailuresModel")

## Model Evaluation (Linear Regression)
There are many ways that an ML model could be refined and improved.  Here we are only interested in understanding whether the model fits the data.


In [14]:
summaryInfo = linearRegModel.evaluate(trainingDf)
print ("Mean Absolute Error", summaryInfo.meanAbsoluteError, "Residuals", summaryInfo.residuals)

Mean Absolute Error 3.308953783115157 Residuals DataFrame[residuals: double]


In [15]:
linearRegModel.transform(trainingDf).show()

+--------------------+-----+------------------+
|            features|count|        prediction|
+--------------------+-----+------------------+
|(29,[12,27],[1.0,...|   11|18.263108126586722|
|(29,[13,27],[1.0,...|   33|19.685012949148994|
|(29,[14,27],[1.0,...|   15|20.270245530353453|
|(29,[15,27],[1.0,...|   24|20.304191118372746|
|(29,[16,27],[1.0,...|   11| 18.60143897190655|
|(29,[17,27],[1.0,...|   10|15.664808076518153|
|(29,[18,27],[1.0,...|   14|12.627661091386695|
|(29,[19,27],[1.0,...|   11|10.307756618950208|
|(29,[20,27],[1.0,...|    5| 8.263558259981334|
|(29,[21,27],[1.0,...|    2| 6.535145274602367|
|(29,[22,27],[1.0,...|    2| 5.419264963727678|
|     (29,[27],[1.0])|    3| 4.169772678038179|
|(29,[0,25],[1.0,1...|    2| 4.339590199932597|
|(29,[1,25],[1.0,1...|    1|  4.06870403903783|
|(29,[2,25],[1.0,1...|    2| 3.855332962540353|
|(29,[3,25],[1.0,1...|    3| 3.005186305103371|
|(29,[4,25],[1.0,1...|    1|3.4023199990892055|
|(29,[5,25],[1.0,1...|    1|2.9789152461

## Training (Logistic Regression)
Although the required prediction is a continuous number, there are possibly so few values that a logistic regression can be used.  Let's try!

In [28]:
logisticReg = LogisticRegression(maxIter=20, regParam=0.001, featuresCol='features', labelCol='count')

logisticRegModel = logisticReg.fit(trainingDf)

logisticRegModel.write().overwrite().save("models/logisticRegressionAuthFailuresModel")

## Model Evaluation (Logistic Regression)
There are many ways that an ML model could be refined and improved.  Here we are only interested in understanding whether the model fits the data.


In [17]:
logisticRegModel.transform(trainingDf).show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|count|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(29,[12,27],[1.0,...|   11|[-4.0761730721365...|[1.34248093842052...|      19.0|
|(29,[13,27],[1.0,...|   33|[-4.0819635222495...|[1.58244653249414...|      18.0|
|(29,[14,27],[1.0,...|   15|[-4.0782723150044...|[1.39467610076262...|      25.0|
|(29,[15,27],[1.0,...|   24|[-4.0794391593501...|[1.48461794255266...|      22.0|
|(29,[16,27],[1.0,...|   11|[-4.0743370372498...|[1.26650973752681...|      19.0|
|(29,[17,27],[1.0,...|   10|[-4.0710260392883...|[1.15161184770848...|      15.0|
|(29,[18,27],[1.0,...|   14|[-4.0611251935154...|[8.75629418542019...|      11.0|
|(29,[19,27],[1.0,...|   11|[-4.0538889215530...|[5.88651530152351...|       9.0|
|(29,[20,27],[1.0,...|    5|[-4.0475646017947...|[5.05084667224068...|       7.0|
|(29,[21,27],[1.

## Training (Random Forest Regression)
Maybe a decision tree / random forest approach might be more successful.

In [26]:
randomForestRegressor = RandomForestRegressor(featuresCol='features', labelCol='count')

randomForestModel = randomForestRegressor.fit(trainingDf)

randomForestModel.write().overwrite().save("models/randomForestAuthFailuresModel")

## Model Evaluation (Random Forest Regression)
There are many ways that an ML model could be refined and improved.  Here we are only interested in understanding whether the model fits the data.

In [27]:
randomForestModel.transform(trainingDf).show()

+--------------------+-----+------------------+
|            features|count|        prediction|
+--------------------+-----+------------------+
|(29,[12,27],[1.0,...|   11|12.091971403530035|
|(29,[13,27],[1.0,...|   33|16.773290187280313|
|(29,[14,27],[1.0,...|   15|14.901076640562056|
|(29,[15,27],[1.0,...|   24| 18.85454716792689|
|(29,[16,27],[1.0,...|   11| 14.23039128613128|
|(29,[17,27],[1.0,...|   10| 9.372821630417725|
|(29,[18,27],[1.0,...|   14| 8.718060315407316|
|(29,[19,27],[1.0,...|   11| 8.718060315407316|
|(29,[20,27],[1.0,...|    5| 8.718060315407316|
|(29,[21,27],[1.0,...|    2| 8.718060315407316|
|(29,[22,27],[1.0,...|    2| 8.282301702394571|
|     (29,[27],[1.0])|    3| 8.718060315407316|
|(29,[0,25],[1.0,1...|    2| 8.077472440375885|
|(29,[1,25],[1.0,1...|    1| 6.058537462694453|
|(29,[2,25],[1.0,1...|    2| 7.333701272081816|
|(29,[3,25],[1.0,1...|    3| 7.791818188774362|
|(29,[4,25],[1.0,1...|    1| 8.142222349101697|
|(29,[5,25],[1.0,1...|    1| 8.503737500